In [13]:
import nltk
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
from spamassassin_client import SpamAssassin
from nltk.tokenize.treebank import TreebankWordDetokenizer
from nltk.corpus import wordnet as wn
from util import run_sa, evaluate

%load_ext autoreload
%autoreload 2

In [16]:
with open('spammy_words.txt', 'r') as f:
    spammy_words = set(f.read().splitlines())
len(list(spammy_words))

232

In [17]:
# just default to getting first synset of word, ordered by popularity, wont guarantee correct usage in context but should be good enough
# could use BERT in this way as well? "A synonym for ___ is ___" prompt?
# get nex
def replace_word(word):
    try: 
        synset = wn.synsets(word)[0] # get most popular synset
        lemmas = synset.lemmas()
        for l in lemmas:
            if word.lower() != l.name().lower():
                return True, l.name().replace("_", " ")
        return False, word
    except:
        return False, word

In [18]:
def poison_word(word):
    return "..".join(word)

In [ ]:
def get_subj_body_tokens(header, body):
    pass

In [19]:
def synonym_replace_attack(email):
    spl = email.split("\n\n", 1)
    header, body = spl[0], spl[1]

    header = spl[0] # first line in subject is always the subject 
    header_spl = header.split("\n", 1)
    remain_header = header_spl[1]

    subj = header_spl[0]

    subj_content = subj.split("Subject: ", 1)[1].strip()
    body_content = spl[1]

    # TOKENIZE
    subj_tokens = nltk.word_tokenize(subj_content)
    body_tokens = nltk.word_tokenize(body_content)

    new_subj_tokens = []
    new_body_tokens = []

    num_potential = 0 # we know there is a spammy word here
    num_successful = 0 # we replaced it with something
    repl = [] # replacement information
    for s_t in subj_tokens:
        w_new = s_t
        if s_t in spammy_words:
            num_potential += 1
            success, new_word = replace_word(s_t)
            w_new = new_word
            if success:
                num_successful += 1
            repl.append((s_t, new_word, success))
        new_subj_tokens.append(w_new)
            

    for b_t in body_tokens:
        w_new = b_t
        if b_t in spammy_words:
            num_potential += 1
            success, new_word = replace_word(b_t)
            w_new = new_word
            if success:
                num_successful += 1
            repl.append((b_t, new_word, success))
        new_body_tokens.append(w_new)

    new_header = "Subject: " + TreebankWordDetokenizer().detokenize(new_subj_tokens) + "\n" + remain_header
    new_body = TreebankWordDetokenizer().detokenize(new_body_tokens)
    new_email = new_header + "\n\n" + new_body

    return num_potential, num_successful, repl, new_email

In [20]:
good_spam_df = pd.read_csv('data/goodSpam/goodSpam.csv')

In [21]:
# executing the attack
syn_repl_ret = [synonym_replace_attack(t) for t in good_spam_df['text']]
# poison_ret

In [22]:
# how many were successful?
pot_successes = sum([1 if t[0] > 0 else 0 for t in syn_repl_ret])
successes = sum([1 if t[1] > 0 else 0 for t in syn_repl_ret])

print("Total emails: ", len(good_spam_df))
print("Potential Replacements (has at least one 'spammy' word): ", pot_successes)
print("Had at least one successful replacement: ", successes)

Total emails:  268
Potential Replacements (has at least one 'spammy' word):  261
Had at least one successful replacement:  248


In [23]:
og_spam_text = good_spam_df['text']
new_spam_text = [t[3] for t in syn_repl_ret]
spam_labels = good_spam_df['label']

# evaluate
spam_inputs = list(zip(new_spam_text, spam_labels))
_ = evaluate(spam_inputs)

1/268 2/268 3/268 4/268 5/268 6/268 7/268 8/268 9/268 10/268 11/268 12/268 13/268 14/268 15/268 16/268 17/268 18/268 19/268 20/268 21/268 22/268 23/268 24/268 25/268 26/268 27/268 28/268 29/268 30/268 31/268 32/268 33/268 34/268 35/268 36/268 37/268 38/268 39/268 40/268 41/268 42/268 43/268 44/268 45/268 46/268 47/268 48/268 49/268 50/268 51/268 52/268 53/268 54/268 55/268 56/268 57/268 58/268 59/268 60/268 61/268 62/268 63/268 64/268 65/268 66/268 67/268 68/268 69/268 70/268 71/268 72/268 73/268 74/268 75/268 76/268 77/268 78/268 79/268 80/268 81/268 82/268 83/268 84/268 85/268 86/268 87/268 88/268 89/268 90/268 91/268 92/268 93/268 94/268 95/268 96/268 97/268 98/268 99/268 100/268 101/268 102/268 103/268 104/268 105/268 106/268 107/268 108/268 109/268 110/268 111/268 112/268 113/268 114/268 115/268 116/268 117/268 118/268 119/268 120/268 121/268 122/268 123/268 124/268 125/268 126/268 127/268 128/268 129/268 130/268 131/268 132/268 133/268 134/268 135/268 136/268 137/268 138/268 139/

/Users/nickhansen/miniconda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nickhansen/miniconda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/nickhansen/miniconda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
